In [0]:
# from google.colab import files
# uploaded = files.upload()

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/nlp_project/data')
sys.path.append('/content/gdrive/My Drive/nlp_project/english_hindi')


In [3]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

!pip install transformers

     |████████████████████████████████| 450kB 3.4MB/s 
     |████████████████████████████████| 870kB 56.1MB/s 
     |████████████████████████████████| 1.0MB 44.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=a6ac6a90e79a5a73b75a5e7104fd78df647d3858a4fcb6d74d8111af90ad7239
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [5]:
len(tokenizer.vocab)
tokens = tokenizer.tokenize('how ARE you ')

print(tokens)
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)


['how', 'AR', '##E', 'you']
[14796, 50884, 11259, 13028]


In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)


# We can get the indexes of the special tokens by converting them using the vocabulary...

# In[7]:


init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# ...or by explicitly getting them from the tokenizer.

# In[8]:


init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

# In[9]:


max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']

print(max_input_length)


# Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

# In[10]:


def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens


# Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.
# 
# We define the label field as before.

# In[87]:


from torchtext import data


[CLS] [SEP] [PAD] [UNK]
101 102 0 100
101 102 0 100
512


In [7]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField()


# We load the data and create the validation splits as before.

# In[89]:


from torchtext import datasets
fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}
train_data, test_data = data.TabularDataset.splits(
                                        path = '/content/gdrive/My Drive/nlp_project/english_hindi/',
                                        train = 'train.txt',
                                        test = 'test.txt',
                                        format = 'tsv',
                                        fields = fields,
                                        skip_header = False
)
# train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# valid_data = train_data
train_data, valid_data = train_data.split(random_state = random.seed(SEED))


# In[90]:


print(vars(train_data[1]))


{'text': [155, 14835, 20429, 877, 13718, 57523, 73354, 20429, 11267, 29698, 54609, 11290, 10140, 10112, 10412, 18867, 20429, 119, 119], 'label': '1'}


In [8]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")


# We can check an example and ensure that the text has already been numericalized.

# In[92]:


print(vars(train_data.examples[6]))


# We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

# In[93]:


tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)


# Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

# In[94]:


LABEL.build_vocab(train_data)


# In[95]:


print(LABEL.vocab.stoi)


# As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

# In[96]:


BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Number of training examples: 9771
Number of validation examples: 4187
Number of testing examples: 1000
{'text': [850, 111192, 15552, 20429, 10569, 100, 865, 20429, 10168, 22888, 40840, 10112, 13375, 58490, 23315, 13971, 10569, 884, 52487, 153, 111192, 66943, 57523, 41013, 10911, 179, 10410, 10174, 40265, 23491, 12141, 13295, 23491, 12141, 13295, 23491, 12141, 13295], 'label': '0'}
['ः', '##अ', '##द', '##्', 'है', '[UNK]', 'क', '##्', 'du', '##ffer', 'insan', '##e', 'full', 'load', '##sha', '##ding', 'है', 'न', '##ये', 'P', '##अ', '##कि', '##स्त', '##न्', 'me', 'k', '##p', '##k', 'मे', 'pou', '##ting', 'face', 'pou', '##ting', 'face', 'pou', '##ting', 'face']
defaultdict(<function _default_unk_index at 0x7f4d2baa6048>, {'1': 0, '2': 1, '0': 2})


In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key=lambda x: len(x.text), 
    batch_size = BATCH_SIZE, 
    device = device)


In [0]:

from transformers import BertTokenizer, BertModel
bert = BertModel.from_pretrained('bert-base-multilingual-cased')


In [0]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        # self.conv = nn.Conv1d(74,74, 3)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
        print(embedded.size())
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        print(hidden.size())
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        print(hidden.size())
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output


In [13]:
HIDDEN_DIM = 256
OUTPUT_DIM = 3
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)


# We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

# In[100]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


# In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

# In[101]:


for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False


# We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

# In[102]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 181,796,355 trainable parameters
The model has 3,942,915 trainable parameters


In [0]:
model

In [0]:

for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())


# In[105]:


criterion = nn.CrossEntropyLoss()


# Place the model and criterion onto the GPU (if available)

# In[106]:


model = model.to(device)
criterion = criterion.to(device)


# Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

# In[107]:




In [0]:

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])


# In[108]:


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


# In[110]:


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [17]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        model_save_name = 'temp_26Jan.pt'
        torch.save(model.state_dict(), model_save_name)
        path = F"/content/gdrive/My Drive/nlp_project/{model_save_name}" 
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



torch.Size([128, 74, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 102, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 117, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 73, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 72, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 121, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 70, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 87, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 98, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 101, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 104, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 98, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.Size([128, 72, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
torch.S

KeyboardInterrupt: ignored

In [1]:
model_save_name = 'temp_26Jan.pt'
path = F"/content/gdrive/My Drive/nlp_project/{model_save_name}" 
torch.save(model.state_dict(), path)

NameError: ignored

In [18]:
path = F"/content/gdrive/My Drive/nlp_project/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [0]:

def test_evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_all_acc = torch.FloatTensor([0,0,0,0,0,0])
    print(epoch_all_acc)
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc,all_acc = test_categorical_accuracy(predictions, batch.label)
            print(all_acc)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_all_acc += all_acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),epoch_all_acc/len(iterator)

def test_categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    count0,count1,count2 = torch.zeros(1),torch.zeros(1),torch.zeros(1)
    total0,total1,total2 = torch.FloatTensor(1),torch.FloatTensor(1),torch.FloatTensor(1)
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    
    for j,i in enumerate(y.cpu().numpy()):
      if i==0:
        count0+=correct[j]
        total0+=1
      elif i==1:
        count1+=correct[j]
        total1+=1
      elif i==2:
        count2+=correct[j]
      else:
        print(i,i==0,i==1,i==2)
        total2+=1
    # print(count0,count1,count2,total0,total1,total2)
    # print([count0/total0,count1/total1,count2/total2])
    # print(torch.FloatTensor([count0/total0,count1/total1,count2/total2]))
    # print(correct.sum() / torch.FloatTensor([y.shape[0]]))
    # print(torch.FloatTensor([count0/total0,count1/total1,count2/total2]))
    print(count0,count1,count2)
    return correct.sum() / torch.FloatTensor([y.shape[0]]),torch.FloatTensor([count0/total0,count1/total1,count2/total2,count0,count1,count2])



In [21]:
# model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc,test_all_acc = test_evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%',test_all_acc)


# ## Inference
# 
# We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

# In[ ]:







tensor([0., 0., 0., 0., 0., 0.])
torch.Size([128, 26, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
tensor([34.]) tensor([36.]) tensor([14.])
tensor([5.8621e-01, 6.7925e-01, 1.2233e+09, 3.4000e+01, 3.6000e+01, 1.4000e+01])
torch.Size([128, 33, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
tensor([23.]) tensor([43.]) tensor([14.])
tensor([4.2593e-01, 8.4314e-01, 1.2233e+09, 2.3000e+01, 4.3000e+01, 1.4000e+01])
torch.Size([128, 39, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
tensor([20.]) tensor([35.]) tensor([10.])
tensor([3.6364e-01, 7.0000e-01, 8.7382e+08, 2.0000e+01, 3.5000e+01, 1.0000e+01])
torch.Size([128, 44, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
tensor([18.]) tensor([31.]) tensor([35.])
tensor([4.2857e-01, 7.9487e-01, 3.0581e+09, 1.8000e+01, 3.1000e+01, 3.5000e+01])
torch.Size([128, 49, 768])
torch.Size([6, 128, 256])
torch.Size([128, 512])
tensor([19.]) tensor([22.]) tensor([26.])
tensor([3.8776e-01, 6.1111e-01, 2.2645e+09, 1.9000e+0

In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    # ind = np.argmax(np.array(prediction))
    # if ind ==0:
    #   print('neutral')
    # elif ind == 1:
    #   print("positive")
    # else:
    #   print("negative")
    print(prediction)
predict_sentiment(model, tokenizer, "This film is terrible")


# In[ ]:


predict_sentiment(model, tokenizer, "This film is great")

In [0]:
while True:
  sent = input('->')
  if sent != '$':
    predict_sentiment(model, tokenizer, sent)
  else:
    break
